In [2]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.1 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125924 sha256=9b506b72bac92cfe8a382f869fc8f7c7bb103692e41e99ada9ac66fdedf8f645
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [3]:
import torch.nn.functional as F

from torch import Tensor
from transformers import AutoTokenizer, AutoModel
import json
from google.colab import drive
import pandas as pd

In [4]:
drive.mount('/content/drive')

post_path="/content/drive/MyDrive/TFM/Instagram.Post.json"

d = pd.read_csv('/content/drive/MyDrive/TFM/SentenceEmbeddings.csv', header=None)


Mounted at /content/drive


In [ ]:
done = set(d.iloc[:, 0].tolist())


In [ ]:
def average_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-small')
model = AutoModel.from_pretrained('intfloat/multilingual-e5-small')

with open(post_path, "r") as input:
  inp = json.load(input)
  ids = []
  caption_batch = []
  i =1
  for line in inp:
    if line["id"]["$numberLong"] not in done:
      ids.append(line["id"]["$numberLong"])
      caption_batch.append(f'query: {line["caption"]}')
      if i %5 == 0:
        try:
          batch_dict = tokenizer(caption_batch, max_length=512, padding=True, truncation=True, return_tensors='pt')
          outputs = model(**batch_dict)
          a = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])

          embeddings = a.detach().numpy()
          df_features = pd.DataFrame(embeddings)
          df_ids = pd.DataFrame(ids)
          merged_df = pd.concat((df_ids, df_features), axis=1)
          merged_df.to_csv('/content/drive/MyDrive/TFM/SentenceEmbeddings.csv',  mode='a', header=False, index=False)

          caption_batch = []
          ids = []
        except:
          print(caption_batch)

      i +=1

  batch_dict = tokenizer(caption_batch, max_length=512, padding=True, truncation=True, return_tensors='pt')
  outputs = model(**batch_dict)
  a = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])
  embeddings = a.detach().numpy()
  df_features = pd.DataFrame(embeddings)
  df_ids = pd.DataFrame(ids)
  merged_df = pd.concat((df_ids, df_features), axis=1)
  merged_df.to_csv('/content/drive/MyDrive/TFM/SentenceEmbeddings.csv',  mode='a', header=False, index=False)


['query: ⚓ #latiendadeammi ⚓ Kit de uñas completo y seguro para tu bebé.👶🏻🍼🧷\n\nY la generación que está por nacer, ALABARÁ A DIOS! SALMO 102-18.🤰🏼🤱🏼❤️\n\nNO TE DEJES CONFUNDIR...\n➡️Sólo lo encuentras en nuestra tienda.\n\n¡¡¡ LA ORIGINAL!!! 🔝💯% ✔\n\n☆ LA TIENDA OFICIAL DE LAS MAMÁS!🤰🤱👶👦👧\n\n♡ ¡Sólo Ternura!❤\n\nY todas las naciones os dirán bienaventurados; porque seréis tierra deseable, dice Jehová de los ejércitos.\nMalaquías 3:12\n\n🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻\n\n➡️Siguenos en Facebook e Instagram ⬅️\n\nhttps://www.instagram.com/invites/contact/?i=1bsf83k5n737i&utm_content=e0yu0yx\n♡\n•\n🌎San José del Nus (Ant).\n•\n🚍Despachos Nacionales. 🇨🇴\n\n☎️3022571857 - 3104573376.\n•\n#TodoParaBebé  #tetero  #mibebé #babyshower #ropabebe #mamaprimeriza #mamafeliz #esperandobebe  #baberospersonalizados #baby #niñosyniñasfelices #pañalera #bebes #bebeencamino #bebeabordo #colorespasteles #regalosoriginales #regalosunicos #niñosfelicespadresfelices #cobijasbebe  #ropaniños #lactanciamaterna #ajuarbebe #a

OSError: ignored

In [5]:
df = pd.read_csv('/content/drive/MyDrive/TFM/SentenceEmbeddings.csv', header=None)

# Drop duplicates based on all columns
df.drop_duplicates(subset=df.columns[0], inplace=True)

df.to_csv('/content/drive/MyDrive/TFM/SentenceEmbeddings.csv',  mode='w', header=False, index=False)

In [ ]:
df.size()

TypeError: ignored